# CS109 - Final Project - Market Predictors

## Exploratory Data Analysis

Using [TA-Lib : Technical Analysis Library](http://ta-lib.org/)  
Also good: [Technical analysis documentation](http://tadoc.org/)

###### Notes:
I started analyzing the data provided by my company and wasn't happy with what our systems provided for several reasons--
- We had no control over the parameters used to create the features/indicators (e.g., specifying how many days used to calculate moving averages, or specifying the time periods for the Aroon Oscillator, etc.)
- It wasn't reproducible. Every data-load for a new index or investment would require a new manual feed. Fetching the data and loading from "scratch" allows us to streamline the entire process.
- The process of retrieving the manual feeds was quite laborious. They say that the best programmers are lazy ones (who write code to accomplish things rather than doing a lot of work).

I noticed that literally all of the historical data/metrics/indicators were derived from the basics (date, open, high, low, close, volume). And the basics are offered for download via API from many sources (like Yahoo Finance (used through matplotlib (!) below)).  

I thought I was going to need to implement the calculations/formulas/coding to re-create all of the features/indicators, but after a quick google search of "technical analysis python," I discovered "TA-Lib", an open-source, multi-platform library (with a python wrapper) that has already implemented more features/indicators than we can use.

To use talib, you'll have to [Install it](http://mrjbq7.github.io/ta-lib/install.html), but it is FAST and works VERY well.

- Windows user: it's a pain to compile ta-lib if environment isn't setup correctly. Get [compiled binary](http://www.lfd.uci.edu/~gohlke/pythonlibs/) and run "pip install TA_Lib-0.4.9-cp27-none-win_amd64.whl". 

You can see the flow of my explorations below, culminating in one pandas dataframe containing five full years of daily data for symbol "UXI" (ProShares Ultra Industrials--chosen at random) saved to a csv file ("UXI.csv").

There's still LOTS to do, including:
- adding signal strategies using some of the trend data
- tweaking the parameters for all of the indicators we intend to use
- adding (there are many more available) or removing features/indicators
- standardizing/normalizing the data
- etc...

But I'm planning to focus on HW5 for the next few days, so I wanted to get this into your hands as early as possible. Also--I'm writing these notes more as a diary (before I forget what I've done and the reason for doing it) and to communicate the status to you--I don't intend for this to be in the final product (as is, anyway!).

I've provided several links in this document--well worth exploring.
See you Saturday @ 8:30pm.


In [1]:
import talib
import datetime
import numpy as np
import matplotlib.colors as colors
import matplotlib.finance as finance
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager

In [2]:
#Examples playing around with talib
#from: http://mrjbq7.github.io/ta-lib/index.html

from talib import MA_Type

#upper, middle, lower = talib.BBANDS(close, matype=MA_Type.T3)

#momentum = talib.MOM(close, timeperiod=5)

# note that all ndarrays must be the same length!
inputs = {
    'open': np.random.random(100),
    'high': np.random.random(100),
    'low': np.random.random(100),
    'close': np.random.random(100),
    'volume': np.random.random(100)
}
input_arrays = {
    'open': np.random.random(100),
    'high': np.random.random(100),
    'low': np.random.random(100),
    'close': np.random.random(100),
    'volume': np.random.random(100)
}

from talib import abstract
sma = abstract.SMA
sma = abstract.Function('sma')

from talib.abstract import *
output = SMA(input_arrays, timeperiod=25) # calculate on close prices by default
output = SMA(input_arrays, timeperiod=25, price='open') # calculate on opens
upper, middle, lower = BBANDS(input_arrays, 20, 2, 2)
slowk, slowd = STOCH(input_arrays, 5, 3, 0, 3, 0) # uses high, low, close by default
slowk, slowd = STOCH(input_arrays, 5, 3, 0, 3, 0, prices=['high', 'low', 'open'])

### Here are all the functions (and functions by group) offered through talib

#### Many details on each technical analysis indicator: [SparkCharts](http://stockcharts.com/school/doku.php?id=chart_school:technical_indicators)

In [3]:
#talib functions
print talib.get_functions()
print talib.get_function_groups()

['ATR', 'NATR', 'TRANGE', 'BBANDS', 'DEMA', 'EMA', 'HT_TRENDLINE', 'KAMA', 'MA', 'MAMA', 'MAVP', 'MIDPOINT', 'MIDPRICE', 'SAR', 'SAREXT', 'SMA', 'T3', 'TEMA', 'TRIMA', 'WMA', 'BETA', 'CORREL', 'LINEARREG', 'LINEARREG_ANGLE', 'LINEARREG_INTERCEPT', 'LINEARREG_SLOPE', 'STDDEV', 'TSF', 'VAR', 'ADX', 'ADXR', 'APO', 'AROON', 'AROONOSC', 'BOP', 'CCI', 'CMO', 'DX', 'MACD', 'MACDEXT', 'MACDFIX', 'MFI', 'MINUS_DI', 'MINUS_DM', 'MOM', 'PLUS_DI', 'PLUS_DM', 'PPO', 'ROC', 'ROCP', 'ROCR', 'ROCR100', 'RSI', 'STOCH', 'STOCHF', 'STOCHRSI', 'TRIX', 'ULTOSC', 'WILLR', 'CDL2CROWS', 'CDL3BLACKCROWS', 'CDL3INSIDE', 'CDL3LINESTRIKE', 'CDL3OUTSIDE', 'CDL3STARSINSOUTH', 'CDL3WHITESOLDIERS', 'CDLABANDONEDBABY', 'CDLADVANCEBLOCK', 'CDLBELTHOLD', 'CDLBREAKAWAY', 'CDLCLOSINGMARUBOZU', 'CDLCONCEALBABYSWALL', 'CDLCOUNTERATTACK', 'CDLDARKCLOUDCOVER', 'CDLDOJI', 'CDLDOJISTAR', 'CDLDRAGONFLYDOJI', 'CDLENGULFING', 'CDLEVENINGDOJISTAR', 'CDLEVENINGSTAR', 'CDLGAPSIDESIDEWHITE', 'CDLGRAVESTONEDOJI', 'CDLHAMMER', 'CDLHANGI

In [4]:
#Example of fetching Yahoo historical data
#from: http://matplotlib.org/examples/pylab_examples/finance_work2.html

import pandas as pd
startdate = datetime.date(2008, 1, 1)
today = enddate = datetime.date(2014, 12, 31)#datetime.date.today()
ticker = 'IYR'   # '^GSPC'

fh = finance.fetch_historical_yahoo(ticker, startdate, enddate)
# a numpy record array with fields: date, open, high, low, close, volume, adj_close)

r = mlab.csv2rec(fh)
fh.close()

# this sort always crashes my ipython kernel, not sure why, will do the sort in dataframe below
#r.sort()

In [6]:
#Copy to pandas dataframe

# sort dataframe here instead of r.sort()
df = pd.DataFrame(data = r, columns=['date', 'open', 'high', 'low', 'close', 'volume', 'adj_close']).sort('date')
df['volume'] = df['volume'].astype(float)
df.head()

,date,open,high,low,close,volume,adj_close
1762,2008-01-02,65.730003,66.550003,64.809998,65.150002,7140100,46.712839
1761,2008-01-03,65.650002,65.900002,63.040001,63.040001,7858800,45.199959
1760,2008-01-04,62.320000,63.000000,61.009998,61.040001,11329300,43.765950
1759,2008-01-07,61.759998,62.070000,60.060001,61.610001,8651900,44.174643
1758,2008-01-08,61.900002,63.360001,59.209999,59.369999,10957900,42.568552


Note: "adj_close" is close price adjusted for dividends and splits.

In [7]:
del df['adj_close']

# How to measure
# First, let's get next day's performance
df['close_1'] = np.roll(df['close'],-1)
df['result_1'] = df['close_1'] > df['close']
df['perf_1'] = (df['close_1']-df['close'])/df['close']

# I think a better measure would be 14 days' performance
df['close_14'] = np.roll(df['close'],-13)
df['result_14'] = df['close_14'] > df['close']
df['perf_14'] = (df['close_14']-df['close'])/df['close']

#An even better measure:
#The Golden Rule of investing is “buy low, sell high.” After analyzing the detail in investment charts, 
#the best performance would be obtained by “buying” when the index is in a trough, and selling when the 
#index is at a peak.
#We manually examined two years of peaks and troughs to come up with an “ideal” in/out scenario.
#We determined that the peaks and troughs could be recognized to a great degree based on the close price 
#versus the moving average close price +/-5 days (within a tolerance).
#Formula: Given “prior tolerance (pt) = 0.2%” and “future tolerance (ft) = 0.5%”
#“Sell” if ((closing price * (1+pt)) > EMA[-5]) 
#	AND (closing price * (1+ft) > EMA[+5])
#“Buy” if ((closing price * (1-pt)) < EMA[-5])
# 	AND (closing price * (1-ft) < EMA[+5])
#otherwise, Hold.
#Using this formula, we could create excellent buy/sell “actual” signals that would maximize up-trends 
#(by buying in troughs) and minimize downside exposure (by selling at peaks).
#Our goal would be to match these signals through prior data alone, given that we won’t have the future 
#5-day moving average.
ptol = 0.002
ftol = 0.005
df['ema5p'] = talib.EMA(df['close'].values, timeperiod=5)
df['ema5f'] = np.roll(df['ema5p'],-5)
df['rsell1'] = df['close']*(1+ptol)>df['ema5p']
df['rsell2'] = df['close']*(1+ftol)>df['ema5f']
#df['rsell'] = df['rsell1'] & df['rsell2']
df['rbuy1'] = df['close']*(1-ptol)<df['ema5p']
df['rbuy2'] = df['close']*(1-ftol)<df['ema5f']
df['results'] = np.where(df['rsell1']&df['rsell2'], -1, np.where(df['rbuy1']&df['rbuy2'], 1, 0))

del df['ema5p']
del df['ema5f']
del df['rsell1']
del df['rsell2']
del df['rbuy1']
del df['rbuy2']

df.head(20)

,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,result_14,perf_14,results
1762,2008-01-02,65.730003,66.550003,64.809998,65.150002,7140100,63.040001,False,-0.032387,61.009998,False,-0.063546,0
1761,2008-01-03,65.650002,65.900002,63.040001,63.040001,7858800,61.040001,False,-0.031726,66.000000,True,0.046954,0
1760,2008-01-04,62.320000,63.000000,61.009998,61.040001,11329300,61.610001,True,0.009338,64.709999,True,0.060124,0
1759,2008-01-07,61.759998,62.070000,60.060001,61.610001,8651900,59.369999,False,-0.036358,63.810001,True,0.035708,0
1758,2008-01-08,61.900002,63.360001,59.209999,59.369999,10957900,60.540001,True,0.019707,66.180000,True,0.114704,1
1757,2008-01-09,59.490002,60.599998,57.290001,60.540001,16010500,61.340000,True,0.013214,65.779999,True,0.086554,1
1756,2008-01-10,59.110001,62.439999,58.689999,61.340000,16104400,61.240002,False,-0.001630,64.239998,True,0.047277,0
1755,2008-01-11,60.799999,62.299999,59.700001,61.240002,11200200,61.070000,False,-0.002776,65.769997,True,0.073971,0
1754,2008-01-14,61.820000,62.310001,59.720001,61.070000,8389300,59.480000,False,-0.026036,68.330002,True,0.118880,0
1753,2008-01-15,59.990002,60.419998,59.200001,59.480000,11132200,60.250000,True,0.012946,67.269997,True,0.130968,1


In [7]:
#Create new fields for various technical indicators directly from talib

#Bolinger Bands

#Developed by John Bollinger, Bollinger Bands® are volatility bands placed above and below a moving average. 
#Volatility is based on the standard deviation, which changes as volatility increases and decreases. 
#The bands automatically widen when volatility increases and narrow when volatility decreases.

df['bb_upper'], df['bb_middle'], df['bb_lower'] = talib.BBANDS(df['close'].values, timeperiod=20,
                                                              nbdevup=2, nbdevdn=2, matype=0)
df['bb_pct'] = (df['close'] - df['bb_lower'])/(df['bb_upper'] - df['bb_lower'])
df['bb_bandwidth'] = ( (df['bb_upper'] - df['bb_lower']) / df['bb_middle']) * 100

#Bollinger BandWidth is best known for identifying The Squeeze. This occurs when volatility falls to a very 
#low level, as evidenced by the narrowing bands. The upper and lower bands are based on the standard deviation, 
#which is a measure of volatility. The bands narrow as price flattens or moves within a relatively narrow range. 
#The theory is that periods of low volatility are followed by periods of high volatility. Relatively narrow 
#BandWidth (a.k.a. the Squeeze) can foreshadow a significant advance or decline. After a Squeeze, a price surge 
#and subsequent band break signal the start of a new move. A new advance starts with a Squeeze and subsequent 
#break above the upper band. A new decline starts with a Squeeze and subsequent break below the lower band.

df['bb_squeeze'] = df['bb_bandwidth'] < (pd.rolling_mean(df['bb_bandwidth'].values, window=200)*0.4)
df['bb_signalup'] = df['close'] > df['bb_upper']
df['bb_signaldn'] = df['close'] < df['bb_lower']
df['bb_signalt'] = (np.where(df['bb_squeeze'], np.where(df['bb_signalup'], 1, np.where(df['bb_signaldn'], -1, 0)), 0))

#Can somebody fix this-- this same logic needs to be used many places below...
#bb_signal should be 1 (buy) or -1 (sell). A "0" is a "hold" and should be replaced with the last 1 or -1.
df['bb_signal'] = np.where(df['bb_signalt']==0,np.roll(df['bb_signalt'],1),df['bb_signalt'])

#del df['bb_signalt']

In [8]:
df[df.date>datetime.date(2013, 3, 25)].head(10)

,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,...,bb_upper,bb_middle,bb_lower,bb_pct,bb_bandwidth,bb_squeeze,bb_signalup,bb_signaldn,bb_signalt,bb_signal
1316,2013-03-26,68.629997,69.110001,68.629997,69.050003,8879000,69.160004,True,0.001593,70.790001,...,69.525933,68.829000,68.132068,0.658554,2.025114,False,False,False,0,0
1317,2013-03-27,68.820000,69.209999,68.680000,69.160004,5747800,69.480003,True,0.004627,71.910004,...,69.508566,68.881501,68.254435,0.722069,1.820707,True,False,False,0,0
1318,2013-03-28,69.260002,69.550003,69.029999,69.480003,10163700,69.699997,True,0.003166,71.040001,...,69.524288,68.949501,68.374714,0.961477,1.667269,True,False,False,0,0
1319,2013-04-01,69.510002,69.699997,69.160004,69.699997,10676100,69.879997,True,0.002582,71.040001,...,69.591337,69.022000,68.452664,1.095427,1.649724,True,True,False,1,1
1320,2013-04-02,69.959999,70.120003,69.720001,69.879997,7683800,69.510002,False,-0.005295,71.860001,...,69.745161,69.078000,68.410840,1.101052,1.931614,False,True,False,0,1
1321,2013-04-03,70.110001,70.110001,69.279999,69.510002,5906600,70.529999,True,0.014674,71.959999,...,69.776718,69.089000,68.401283,0.806086,1.990816,False,False,False,0,0
1322,2013-04-04,69.589996,70.610001,69.480003,70.529999,15045800,70.720001,True,0.002694,72.379997,...,70.094083,69.164000,68.233917,1.234342,2.689500,False,True,False,0,0
1323,2013-04-05,69.889999,70.720001,69.750000,70.720001,9219300,71.379997,True,0.009333,72.610001,...,70.393323,69.261000,68.128678,1.144251,3.269726,False,True,False,0,0
1324,2013-04-08,70.730003,71.449997,70.470001,71.379997,9494700,71.410004,True,0.000420,72.519997,...,70.829747,69.389000,67.948253,1.190960,4.152667,False,True,False,0,0
1325,2013-04-09,71.480003,71.650002,71.010002,71.410004,6504500,71.830002,True,0.005882,72.190002,...,71.184565,69.505001,67.825436,1.067112,4.832932,False,True,False,0,0


In [9]:
#Moving averages smooth the price data to form a trend following indicator. They do not predict price 
#direction, but rather define the current direction with a lag. Moving averages lag because they are based 
#on past prices.

#Exponential Moving Average
#Exponential moving averages have less lag and are therefore more sensitive to recent prices - 
#and recent price changes. Exponential moving averages will turn before simple moving averages.

df['ema50'] = talib.EMA(df['close'].values, timeperiod=50)
df['ema150'] = talib.EMA(df['close'].values, timeperiod=150)
df['ema200'] = talib.EMA(df['close'].values, timeperiod=200)

#Trend Identification
#The direction of the moving average conveys important information about prices. 
#A rising moving average shows that prices are generally increasing. 
#A falling moving average indicates that prices, on average, are falling.

#Price Crossovers
#Moving averages can be used to generate signals with simple price crossovers. 
#A bullish signal is generated when prices move above the moving average. 
#A bearish signal is generated when prices move below the moving average.

df['ema_signal1'] = df['close'] > df['ema150']

#Double Crossovers
#Two moving averages can be used together to generate crossover signals. 
#In Technical Analysis of the Financial Markets, John Murphy calls this the “double crossover method”. 
#Double crossovers involve one relatively short moving average and one relatively long moving average. 
#As with all moving averages, the general length of the moving average defines the timeframe for the system. 
#A system using a 50-day SMA and 200-day SMA would be deemed medium-term, perhaps even long-term.
#A bullish crossover occurs when the shorter moving average crosses above the longer moving average. 
#This is also known as a golden cross. A bearish crossover occurs when the shorter moving average crosses 
#below the longer moving average. This is known as a dead cross.

df['ema_signal2'] = df['ema50'] > df['ema200']


#Double Exponential Moving Average
#df['dema50'] = talib.DEMA(df['close'].values, timeperiod=50)
#df['dema100'] = talib.DEMA(df['close'].values, timeperiod=100)
#df['dema200'] = talib.DEMA(df['close'].values, timeperiod=200)

#Weighted Moving Average
#df['wma50'] = talib.WMA(df['close'].values, timeperiod=50)
#df['wma100'] = talib.WMA(df['close'].values, timeperiod=100)
#df['wma200'] = talib.WMA(df['close'].values, timeperiod=200)

#Simple Moving Average
#df['sma50'] = talib.SMA(df['close'].values, timeperiod=50)
#df['sma100'] = talib.SMA(df['close'].values, timeperiod=100)
#df['sma200'] = talib.SMA(df['close'].values, timeperiod=200)

#Moving Average Shell
#df['ma50'] = talib.MA(df['close'].values, timeperiod=50, matype=0)
#df['ma100'] = talib.MA(df['close'].values, timeperiod=100, matype=0)
#df['ma200'] = talib.MA(df['close'].values, timeperiod=200, matype=0)


In [10]:
#Kaufman Adaptive Moving Average
#Developed by Perry Kaufman, Kaufman's Adaptive Moving Average (KAMA) is a moving average designed 
#to account for market noise or volatility. KAMA will closely follow prices when the price swings are 
#relatively small and the noise is low. KAMA will adjust when the price swings widen and follow prices 
#from a greater distance. This trend-following indicator can be used to identify the overall trend, 
#time turning points and filter price movements.

#Chartists can use KAMA like any other trend following indicator, such as a moving average. 
#Chartists can look for price crosses, directional changes and filtered signals.

df['kama50'] = talib.KAMA(df['close'].values, timeperiod=50)
df['kama150'] = talib.KAMA(df['close'].values, timeperiod=150)
df['kama200'] = talib.KAMA(df['close'].values, timeperiod=200)

df['kama_signal1'] = df['close'] > df['kama150']
df['kama_signal2'] = df['kama50'] > df['kama200']

In [11]:
#Parabolic SAR
#Developed by Welles Wilder, the Parabolic SAR refers to a price-and-time-based trading system. 
#Wilder called this the “Parabolic Time/Price System.” SAR stands for “stop and reverse,” which is the 
#actual indicator used in the system. SAR trails price as the trend extends over time. The indicator is 
#below prices when prices are rising and above prices when prices are falling. In this regard, the indicator 
#stops and reverses when the price trend reverses and breaks above or below the indicator.

df['sar'] = talib.SAR(df['high'].values, df['low'].values, acceleration=0.02, maximum=0.2)
df['sar_signal'] = df['close'] > df['sar']

In [12]:
#Average Directional Movement Index
#The Average Directional Index (ADX), Minus Directional Indicator (-DI) and Plus Directional Indicator (+DI) 
#represent a group of directional movement indicators that form a trading system developed by Welles Wilder. 
#Wilder designed ADX with commodities and daily prices in mind, but these indicators can also be applied to stocks. 
#The Average Directional Index (ADX) measures trend strength without regard to trend direction. The other two 
#indicators, Plus Directional Indicator (+DI) and Minus Directional Indicator (-DI), complement ADX by defining 
#trend direction. Used together, chartists can determine both the direction and strength of the trend.

df['adx'] = talib.ADX(df['high'].values, df['low'].values, df['close'].values, timeperiod=14)
df['plus_di'] = talib.PLUS_DI(df['high'].values, df['low'].values, df['close'].values, timeperiod=14)
df['minus_di'] = talib.MINUS_DI(df['high'].values, df['low'].values, df['close'].values, timeperiod=14)

df['adx_trend'] = df['adx'] > 22
df['adx_direction'] = df['plus_di'] > df['minus_di']
df['adx_signal'] = np.where(df['adx_trend'], np.where(df['adx_direction'],1,-1), 0)


In [13]:
#Aroon
#Developed by Tushar Chande in 1995, Aroon is an indicator system that determines whether a stock is 
#trending or not and how strong the trend is. The Aroon indicators measure the number of periods since 
#price recorded an x-day high or low. There are two separate indicators: Aroon-Up and Aroon-Down. 
#A 25-day Aroon-Up measures the number of days since a 25-day high. A 25-day Aroon-Down measures the 
#number of days since a 25-day low.

#df['aroon_dn'], df['aroon_up'] = talib.AROON(df['high'].values, df['low'].values, timeperiod=25)

#Aroon Oscillator
#The Aroon Oscillator is the difference between Aroon-Up and Aroon-Down. These two indicators are usually 
#plotted together for easy comparison, but chartists can also view the difference of these two indicators 
#with the Aroon Oscillator. This indicator fluctuates between -100 and +100 with zero as the middle line. 
#A upward trend bias is present when the oscillator is positive, while a downward trend bias exists when the 
#oscillator is negative.

df['aroon_osc'] = talib.AROONOSC(df['high'].values, df['low'].values, timeperiod=25)

df['aroon_signal'] = df['aroon_osc'] >= 0

In [14]:
#Commodity Channel Index
#Developed by Donald Lambert and featured in Commodities magazine in 1980, the Commodity Channel Index (CCI) 
#is a versatile indicator that can be used to identify a new trend or warn of extreme conditions. 
#Lambert originally developed CCI to identify cyclical turns in commodities, but the indicator can successfully 
#applied to indices, ETFs, stocks and other securities. In general, CCI measures the current price level relative 
#to an average price level over a given period of time. CCI is relatively high when prices are far above their 
#average. CCI is relatively low when prices are far below their average. In this manner, CCI can be used to 
#identify overbought and oversold levels.

df['cci'] = talib.CCI(df['high'].values, df['low'].values, df['close'].values, timeperiod=14)

#The definition of overbought or oversold varies for the Commodity Channel Index (CCI). 
#±100 may work in a trading range, but more extreme levels are needed for other situations. 
#±200 is a much harder level to reach and more representative of a true extreme.

df['cci_signal'] = np.where(df['cci'] < -200, 1, np.where(df['cci'] > 200, -1, 0))

In [15]:
#Moving Average Convergence/Divergence
#Developed by Gerald Appel in the late seventies, the Moving Average Convergence/Divergence oscillator 
#(MACD) is one of the simplest and most effective momentum indicators available. The MACD turns two 
#trend-following indicators, moving averages, into a momentum oscillator by subtracting the longer moving 
#average from the shorter moving average. As a result, the MACD offers the best of both worlds: trend 
#following and momentum. The MACD fluctuates above and below the zero line as the moving averages converge, 
#cross and diverge. Traders can look for signal line crossovers, centerline crossovers and divergences to 
#generate signals. 

#Interpretation
#As its name implies, the MACD is all about the convergence and divergence of the two moving averages. 
#Convergence occurs when the moving averages move towards each other. 
#Divergence occurs when the moving averages move away from each other. 
#The shorter moving average (12-day) is faster and responsible for most MACD movements. 
#The longer moving average (26-day) is slower and less reactive to price changes in the underlying security.

#The MACD Line oscillates above and below the zero line, which is also known as the centerline. 
#These crossovers signal that the 12-day EMA has crossed the 26-day EMA. 
#The direction, of course, depends on the direction of the moving average cross. 
#Positive MACD indicates that the 12-day EMA is above the 26-day EMA. 
#Positive values increase as the shorter EMA diverges further from the longer EMA. 
#This means upside momentum is increasing. 
#Negative MACD values indicates that the 12-day EMA is below the 26-day EMA. 
#Negative values increase as the shorter EMA diverges further below the longer EMA. 
#This means downside momentum is increasing.

#Signal line crossovers are the most common MACD signals. 
#The signal line is a 9-day EMA of the MACD Line. As a moving average of the indicator, 
#it trails the MACD and makes it easier to spot MACD turns. A bullish crossover occurs 
#when the MACD turns up and crosses above the signal line. A bearish crossover occurs when 
#the MACD turns down and crosses below the signal line. Crossovers can last a few days or 
#a few weeks, it all depends on the strength of the move.

df['macd'], df['macd_sigline'], df['macd_hist'] = talib.MACD(df['close'].values,
                                                           fastperiod=12, slowperiod=26, signalperiod=9)
df['macd_signal'] = df['macd_hist'] > 0

In [16]:
#Percentage Price Oscillator
#The Percentage Price Oscillator (PPO) is a momentum oscillator that measures the difference between two 
#moving averages as a percentage of the larger moving average. As with its cousin, MACD, the Percentage 
#Price Oscillator is shown with a signal line, a histogram and a centerline. Signals are generated with 
#signal line crossovers, centerline crossovers and divergences. Because these signals are no different 
#than those associated with MACD, this article will focus on a few differences between the two. 
#First, PPO readings are not subject to the price level of the security. Second, PPO readings for different 
#securities can be compared, even when there are large differences in the price.

df['ppo'] = talib.PPO(df['close'].values, fastperiod=12, slowperiod=26, matype=0)

df['ppo_signal'] = df['ppo'] > 0

In [17]:
#Money Flow Index
#The Money Flow Index (MFI) is an oscillator that uses both price and volume to measure buying and selling 
#pressure. Created by Gene Quong and Avrum Soudack, MFI is also known as volume-weighted RSI. MFI starts 
#with the typical price for each period. Money flow is positive when the typical price rises (buying pressure) 
#and negative when the typical price declines (selling pressure). 

#Overbought and oversold levels can be used to identify unsustainable price extremes. 
#Typically, MFI above 80 is considered overbought and MFI below 20 is considered oversold. 
#Strong trends can present a problem for these classic overbought and oversold levels. 
#MFI can become overbought (>80) and prices can simply continue higher when the uptrend is strong. 
#Conversely, MFI can become oversold (<20) and prices can simply continue lower when the downtrend is strong.

df['mfi'] = talib.MFI(df['high'].values, df['low'].values, df['close'].values, df['volume'].values, timeperiod=14)

df['mfi_signal'] = np.where(df['mfi'] < 20, 1, np.where(df['mfi'] > 80, -1, 0))

In [18]:
#Rate of change percentage
#The Rate-of-Change (ROC) indicator, which is also referred to as simply Momentum, is a pure momentum 
#oscillator that measures the percent change in price from one period to the next. The ROC calculation 
#compares the current price with the price “n” periods ago. The plot forms an oscillator that fluctuates 
#above and below the zero line as the Rate-of-Change moves from positive to negative.

df['roc'] = 100*talib.ROCP(df['close'].values, timeperiod=10)

#The 20-day Rate-of-Change indicator sets overbought at +10% and oversold at -10%.

df['roc_signal'] = np.where(df['roc'] < -10, 1, np.where(df['roc'] > 10, -1, 0))

In [19]:
#Relative Strength Index
#Developed J. Welles Wilder, the Relative Strength Index (RSI) is a momentum oscillator that measures the 
#speed and change of price movements. RSI oscillates between zero and 100. Traditionally, and according to 
#Wilder, RSI is considered overbought when above 70 and oversold when below 30. Signals can also be generated 
#by looking for divergences, failure swings and centerline crossovers. RSI can also be used to identify the 
#general trend.

df['rsi'] = talib.RSI(df['close'].values, timeperiod=14)

df['rsi_signal'] = np.where(df['rsi'] < 30, 1, np.where(df['rsi'] > 70, -1, 0))

In [20]:
#Ultimate Oscillator
#Developed by Larry Williams in 1976 and featured in Stocks & Commodities Magazine in 1985, the Ultimate 
#Oscillator is a momentum oscillator designed to capture momentum across three different time frames. 
#The multiple time frame objective seeks to avoid the pitfalls of other oscillators. Many momentum oscillators 
#surge at the beginning of a strong advance and then form bearish divergence as the advance continues. 
#This is because they are stuck with one time frame. The Ultimate Oscillator attempts to correct this fault 
#by incorporating longer time frames into the basic formula. Williams identified a buy signal a based on a 
#bullish divergence and a sell signal based on a bearish divergence.

df['ult_osc'] = talib.ULTOSC(df['high'].values, df['low'].values, df['close'].values, 
                            timeperiod1=7, timeperiod2=14, timeperiod3=28)

#Simplified, the ultimate oscillator is considered overbought when above 70 and oversold when below 30.

df['ult_signal'] = np.where(df['ult_osc'] < 30, 1, np.where(df['ult_osc'] > 70, -1, 0))

In [21]:
#Williams' %R
#Developed by Larry Williams, Williams %R is a momentum indicator that is the inverse of the Fast 
#Stochastic Oscillator. Also referred to as %R, Williams %R reflects the level of the close relative 
#to the highest high for the look-back period. In contrast, the Stochastic Oscillator reflects the level 
#of the close relative to the lowest low. %R corrects for the inversion by multiplying the raw value by -100. 
#As a result, the Fast Stochastic Oscillator and Williams %R produce the exact same lines, only the scaling 
#is different. Williams %R oscillates from 0 to -100.

df['willr'] = talib.WILLR(df['high'].values, df['low'].values, df['close'].values, timeperiod=14)

#Readings from 0 to -20 are considered overbought. Readings from -80 to -100 are considered oversold.

df['wr_signal'] = np.where(df['willr'] < -80, 1, np.where(df['willr'] > -20, -1, 0))

In [22]:
#Chaikin A/D Line
#Developed by Marc Chaikin, the Accumulation Distribution Line is a volume-based indicator designed to 
#measure the cumulative flow of money into and out of a security. Chaikin originally referred to the indicator 
#as the Cumulative Money Flow Line. As with cumulative indicators, the Accumulation Distribution Line is a 
#running total of each period's Money Flow Volume. First, a multiplier is calculated based on the relationship 
#of the close to the high-low range. Second, the Money Flow Multiplier is multiplied by the period's volume to 
#come up with a Money Flow Volume. A running total of the Money Flow Volume forms the Accumulation Distribution 
#Line. Chartists can use this indicator to affirm a security's underlying trend or anticipate reversals when 
#the indicator diverges from the security price.

#df['ad'] = talib.AD(df['high'].values, df['low'].values, df['close'].values, df['volume'].values)

#Chaikin A/D Oscillator
#Trend confirmation is a pretty straight-forward concept. An uptrend in the Accumulation Distribution Line 
#reinforces an uptrend on the price chart and vice versa.

df['ad_osc'] = talib.ADOSC(df['high'].values, df['low'].values, df['close'].values, df['volume'].values,
                          fastperiod=3, slowperiod=10)

df['ad_signal'] = df['ad_osc'] > 0

In [23]:
#On Balance Volume
#On Balance Volume (OBV) measures buying and selling pressure as a cumulative indicator that adds volume 
#on up days and subtracts volume on down days. OBV was developed by Joe Granville and introduced in his 1963 
#book, Granville's New Key to Stock Market Profits. It was one of the first indicators to measure positive 
#and negative volume flow. Chartists can look for divergences between OBV and price to predict price movements 
#or use OBV to confirm price trends.

#df['obv'] = talib.OBV(df['close'].values, df['volume'].values)


In [24]:
#Stochastic
#Developed by George C. Lane in the late 1950s, the Stochastic Oscillator is a momentum indicator that shows 
#the location of the close relative to the high-low range over a set number of periods. According to an interview 
#with Lane, the Stochastic Oscillator “doesn't follow price, it doesn't follow volume or anything like that. 
#It follows the speed or the momentum of price. As a rule, the momentum changes direction before price.” As such, 
#bullish and bearish divergences in the Stochastic Oscillator can be used to foreshadow reversals. This was the 
#first, and most important, signal that Lane identified. Lane also used this oscillator to identify bull and bear 
#set-ups to anticipate a future reversal. Because the Stochastic Oscillator is range bound, is also useful for 
#identifying overbought and oversold levels.

df['stoch_slowk'], df['stoch_slowd'] = talib.STOCH(df['high'].values, df['low'].values, df['close'].values, 
                                                   fastk_period=5,
                                                    slowk_period=3,
                                                    slowk_matype=0,
                                                    slowd_period=3,
                                                    slowd_matype=0)

#Traditional settings use 80 as the overbought threshold and 20 as the oversold threshold.

df['sslow_signal'] = np.where(df['stoch_slowk'] < 20, 1, np.where(df['stoch_slowk'] > 80, -1, 0))

In [25]:
#Stochastic Relative Strength Index
#Developed by Tushar Chande and Stanley Kroll, StochRSI is an oscillator that measures the level of RSI 
#relative to its high-low range over a set time period. StochRSI applies the Stochastics formula to RSI values, 
#instead of price values. This makes it an indicator of an indicator. The result is an oscillator that 
#fluctuates between 0 and 1.

df['stoch_fastk'], df['stoch_fastd'] = talib.STOCHRSI(df['close'].values, timeperiod=14,
                                                      fastk_period=5, fastd_period=3, fastd_matype=0)

#In their 1994 book, The New Technical Trader, Chande and Kroll explain that RSI can oscillate between 
#80 and 20 for extended periods without reaching extreme levels. Notice that 80 and 20 are used for overbought 
#and oversold instead of the more traditional 70 and 30. Traders looking to enter a stock based on an overbought 
#or oversold reading in RSI might find themselves continuously on the sidelines. Chande and Kroll developed 
#StochRSI to increase sensitivity and generate more overbought/oversold signals.

df['srsi_signal'] = np.where(df['stoch_fastk'] < 20, 1, np.where(df['stoch_fastk'] > 80, -1, 0))

In [26]:
#1-day Rate-Of-Change (ROC) of a Triple Smooth EMA
#TRIX is a momentum oscillator that displays the percent rate of change of a triple exponentially smoothed 
#moving average. It was developed in the early 1980's by Jack Hutson, an editor for Technical Analysis of 
#Stocks and Commodities magazine. With its triple smoothing, TRIX is designed to filter insignificant price 
#movements. Chartists can use TRIX to generate signals similar to MACD. A signal line can be applied to look 
#for signal line crossovers. A directional bias can be determined with the absolute level. Bullish and bearish 
#divergences can be used to anticipate reversals.

df['trix'] = talib.TRIX(df['close'].values, timeperiod=30)

df['trix_signal'] = df['trix'] > 0

In [27]:
#Derive fields for Support and Resistance

tperiod = 30
lotrade = talib.MIN(df['low'].values, timeperiod=tperiod)
hitrade = talib.MAX(df['high'].values, timeperiod=tperiod)
df['sr_pivotpts'] = (df['high'].values + df['low'].values + df['close'].values)/3
df['sr_res1'] = 2*df['sr_pivotpts'].values - lotrade
df['sr_sup1'] = 2*df['sr_pivotpts'].values - hitrade
df['sr_res2'] = df['sr_pivotpts'].values + (hitrade - lotrade)
df['sr_sup2'] = df['sr_pivotpts'].values - (hitrade - lotrade)
df['sr_res3'] = hitrade + 2*(df['sr_pivotpts'].values - lotrade)
df['sr_sup3'] = lotrade - 2*(hitrade - df['sr_pivotpts'].values)

In [28]:
#Derive Custom Volatility Signal

rangetol1, rangetol2 = 0.005, 0.005
dv = pd.DataFrame(data = r, columns=['date', 'open', 'high', 'low', 'close', 'volume', 'adj_close'])
dv['range'] = dv['high'].values - dv['low'].values
dv['rangepct'] = dv['range'].values/dv['low'].values
dv['prior'] = talib.SUM(dv['close'].values, timeperiod=2) - dv['close'].values
dv['perf'] = (dv['close'].values - dv['prior'].values) / dv['prior'].values
dv['r180'] = pd.rolling_mean(dv['rangepct'].values, window=180)
#dv['vtest1'] = (dv['perf'].values>0 & (dv['rangepct'].values > (dv['r180'].values + rangetol1)))
dv['vtest1a'] = dv['perf'].values > 0
dv['vtest1b'] = (dv['rangepct'].values > (dv['r180'].values + rangetol1))
dv['vtest1'] = dv['vtest1a'] & dv['vtest1b']
#dv['vtest2'] = (dv['perf'].values<0 & (dv['rangepct'].values > (dv['r180'].values + rangetol2)))
dv['vtest2a'] = dv['perf'].values < 0
dv['vtest2b'] = (dv['rangepct'].values > (dv['r180'].values + rangetol2))
dv['vtest2'] = dv['vtest2a'] & dv['vtest2b']
dv['vc'] = dv['vtest1'] | dv['vtest2']
dv['atest1'] = (pd.rolling_sum(dv['vc'], window=3) - dv['vc']) <= 1
dv['atest2'] = pd.rolling_sum(dv['vc'], window=5) == 0.
dv['atest3'] = pd.rolling_sum(dv['vc'], window=15) <= 1.
dv['atest23'] = (dv['atest2'] | dv['atest3'])
dv['in_or_out'] = dv['atest1'] & dv['atest23']
dv['in_or_out_p'] = (pd.rolling_sum(dv['in_or_out'], window=2) - dv['in_or_out']) == 1
dv['in_or_out_e'] = dv['in_or_out'] == dv['in_or_out_p']
dv['cv_signal'] = dv['in_or_out'] & dv['in_or_out_e']

df['cv_signal'] = dv['cv_signal']

In [29]:
df.describe()
df.dtypes
df.head()

,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,...,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
0,2008-01-02,65.730003,66.550003,64.809998,65.150002,7140100,63.040001,False,-0.032387,61.009998,...,NaN,False,65.503334,NaN,NaN,NaN,NaN,NaN,NaN,False
1,2008-01-03,65.650002,65.900002,63.040001,63.040001,7858800,61.040001,False,-0.031726,66.000000,...,NaN,False,63.993335,NaN,NaN,NaN,NaN,NaN,NaN,False
2,2008-01-04,62.320000,63.000000,61.009998,61.040001,11329300,61.610001,True,0.009338,64.709999,...,NaN,False,61.683333,NaN,NaN,NaN,NaN,NaN,NaN,False
3,2008-01-07,61.759998,62.070000,60.060001,61.610001,8651900,59.369999,False,-0.036358,63.810001,...,NaN,False,61.246667,NaN,NaN,NaN,NaN,NaN,NaN,False
4,2008-01-08,61.900002,63.360001,59.209999,59.369999,10957900,60.540001,True,0.019707,66.180000,...,NaN,False,60.646666,NaN,NaN,NaN,NaN,NaN,NaN,False


In [30]:
df.tail()

,date,open,high,low,close,volume,close_1,result_1,perf_1,close_14,...,trix,trix_signal,sr_pivotpts,sr_res1,sr_sup1,sr_res2,sr_sup2,sr_res3,sr_sup3,cv_signal
1758,2014-12-24,77.970001,78.260002,77.430000,77.480003,4049500,77.779999,True,0.003872,61.07,...,0.115843,True,77.723335,80.836669,76.316673,82.243331,73.203339,85.356665,71.796677,False
1759,2014-12-26,77.650002,77.970001,77.629997,77.779999,2903000,78.120003,True,0.004371,59.48,...,0.116547,True,77.793332,80.976664,76.456668,82.313328,73.273336,85.496660,71.936672,False
1760,2014-12-29,77.699997,78.440002,77.480003,78.120003,3913300,78.040001,False,-0.001024,60.25,...,0.117224,True,78.013336,81.416671,76.896675,82.533332,73.493340,85.936667,72.376679,False
1761,2014-12-30,77.889999,78.410004,77.830002,78.040001,7674500,76.839996,False,-0.015377,59.52,...,0.117809,True,78.093336,81.576670,77.056674,82.613332,73.573340,86.096666,72.536678,False
1762,2014-12-31,78.260002,78.620003,76.760002,76.839996,9269300,65.150002,False,-0.152134,59.02,...,0.117844,True,77.406667,80.203333,75.683337,81.926663,72.886671,84.723329,71.163341,False


In [31]:
#Delete oldest data (used only for accurate rolling periods)
df = df[df.date>datetime.date(2010, 1, 1)]
tofile = "data/"+ticker+".csv"
print tofile
#Save df to csv
df.to_csv(tofile, index=False)

data/IYR.csv


In [8]:
def calculate_commission(price, quantity):
    #fix rate pricing from IB including all exchange and regularotry fees
    #https://www.interactivebrokers.com/en/index.php?f=1590&p=stocks1&ns=T
    return np.minimum(np.maximum(0.005*quantity, 1), price*quantity*0.005)

'''
input:
signal_column: name of the column that indicates the signal,  1 for long signal or -1 for short signal
price_column: name of the column responding to order execution price, could be open or close
include_commision: deduct commission for each transaction if True, default to True

output: 
balance, profit, ROI

NOTE: this is LONG only evaluation, SHORT is not implemented yet. SHORT position will involve margin, so we need to decide 
later.

'''
def evaluate_profit(df, start_date, end_date, intitial_balance, signal_column, price_column, include_commission=True):
    df = df.sort('date')
    df = df[df.date>=start_date][df.date<=end_date]
    
    balance = intitial_balance
    current_position = 0 #1 for long position, -1 for short, 0 for closed position or no position
    pos_quantity = 0 # current position quantity
    pos_price = 0 #current position price

    #iterate from first to n-1 row, as we want to close our position on the last day.
    for index, row in df.head(-1).iterrows():    
        new_position = row[signal_column]
        current_price = row[price_column]
        
        
        # receive signal to reverse current position
        if current_position != new_position:
            # bullish market: close short position and open a long position
            if new_position == 1:
                #open a long position (short position not implemented yet)
                pos_quantity = np.floor_divide(balance, current_price) 
                balance = balance - current_price * pos_quantity
                if include_commission:
                    balance = balance - calculate_commission(current_price, pos_quantity)
                
                pos_price = current_price
                current_position = new_position
                #print "long ", row['date'], balance, pos_quantity, pos_price
            
            # bearish market: close long position and open a short position
            else:
                #close the long position (short position not implemented yet)
                balance = balance + current_price * pos_quantity
                if include_commission:
                    balance = balance - calculate_commission(current_price, pos_quantity)
                    
                pos_quantity = 0
                pos_price = 0
                current_position = new_position
                #print "close", row['date'], balance, pos_quantity, pos_price
            
        #hold position and do nothing
        else:
            #print "hold ", row['date'], balance, pos_quantity, pos_price
            pass
            
            
    #found open position on the last day, let's close it
    if pos_quantity != 0:
        # get price of last day
        current_price = df.tail(1).iloc[0][price_column]
        balance = balance + pos_quantity * current_price
        if include_commission:
            balance = balance - calculate_commission(current_price, pos_quantity)
    
    profit = balance - intitial_balance
    ROI = profit/intitial_balance
    
    return  balance, profit, ROI

In [23]:
'''
Example for evaluate_profit() 

'''

start_date = datetime.date(2008, 1, 1)
end_date = datetime.date(2008, 12, 31)
df_test = df.copy()
df_test['result_14'] = df['result_14']*2-1 # convert signal from true/false to 1/0
df_test['result_1'] = df['result_1']*2-1 # convert signal from true/false to 1/0

#use result_14 as the signal column, close price as execution price, initial fund $10,000, include commmision & fee
balance, profit, ROI = evaluate_profit(df_test, start_date, end_date, 10000, 'result_14', 'close', True)
print "ROI over {0} days: {1:.2f}%".format((end_date-start_date).days, ROI*100)


ROI over 365 days: 140.67%
